In [101]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [102]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [103]:
####################### NASA Mars News ##########################

In [104]:
# Mars news URL of page to be scraped
news_url = 'https://redplanetscience.com'

browser.visit(news_url)

html = browser.html

news_soup = BeautifulSoup(html, 'html.parser')

In [105]:
# Retrieve the latest news title and paragraph
news_title = news_soup.find_all('div', class_='content_title')[0].text
news_p = news_soup.find_all('div', class_='article_teaser_body')[0].text

print(news_title)
print("--------------------------------------------------------------------")
print(news_p)

NASA Mars Mission Connects With Bosnian and Herzegovinian Town
--------------------------------------------------------------------
A letter from NASA was presented to the mayor of Jezero, Bosnia-Herzegovina, honoring the connection between the town and Jezero Crater, the Mars 2020 rover landing site.


In [106]:
######################## JPL Mars Space Images - Featured Image ##########################

In [107]:
# Mars Image to be scraped
spaceimages_mars_url = 'https://spaceimages-mars.com'
images_url = 'https://spaceimages-mars.com/image/featured/mars2.jpg'

browser.visit(images_url)

html = browser.html

images_soup = BeautifulSoup(html, 'html.parser')

In [108]:
# Retrieve featured image link
relative_image_path = images_soup.find_all('img')[0]["src"]
featured_image_url = spaceimages_mars_url + relative_image_path
print(featured_image_url)

https://spaceimages-mars.comhttps://spaceimages-mars.com/image/featured/mars2.jpg


In [109]:
#######################  Mars Facts ##########################
url_facts='https://galaxyfacts-mars.com/'
tables = pd.read_html(url_facts)
df=tables[0]
df.columns=['Mars - Earth Comparison','Mars','Earth']
df

,Mars - Earth Comparison,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [110]:

Mars_only_df=df[["Mars - Earth Comparison","Mars"]]
Mars_only_df=Mars_only_df[1:]
Mars_only_df=Mars_only_df.rename(columns={"Mars - Earth Comparison":"Info","Mars":"Data"})
Mars_only_df.head()

,Info,Data
1,Diameter:,"6,779 km"
2,Mass:,6.39 × 10^23 kg
3,Moons:,2
4,Distance from Sun:,"227,943,824 km"
5,Length of Year:,687 Earth days


In [111]:
html_table = Mars_only_df.to_html()
fact_table=Mars_only_df.to_html('table.html')
print(fact_table)

None


In [112]:
######################## Mars Hemispheres ##########################

In [113]:
hemis_url= 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemis_url)
soup = BeautifulSoup(browser.html, 'html.parser')

hemisphere_image_urls = []

results_hemis = soup.find_all('div', class_='item')

for item in results_hemis:
    title_hemis = item.h3.text
    first_url = hemis_url[:30] + item.find('a', class_='itemLink')['href']
    browser.visit(first_url)
    soup = BeautifulSoup(browser.html, 'html.parser')
    #time.sleep(1)
    final_url = hemis_url[:30] + soup.find('img', class_='wide-image')['src']
    hemisphere_image_urls .append({'title': title_hemis, 'img_url':final_url})
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [114]:
# Create dictionary for all info scraped from sources above
mars_dict={
    "news_title":news_title,
    "news_p":news_p,
    "featured_image_url":featured_image_url,
    "fact_table":fact_table,
    "hemisphere_images":hemisphere_image_urls
}

In [115]:
browser.quit()